# Using llama-parse with AstraDB

In this notebook, we show a basic RAG-style example that uses `llama-parse` to parse a PDF document, store the corresponding document into a vector store (`AstraDB`) and finally, perform some basic queries against that store. The notebook is modeled after the quick start notebooks and hence is meant as a way of getting started with `llama-parse`, backed by a vector database.

### Requirements

In [ ]:
# First, install the required dependencies
!pip install --quiet ragstack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Configuration

In [ ]:
import os
from getpass import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass("Enter your Llama Index Cloud API Key:")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("Enter your Astra DB API Endpoint: ")
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("Enter your Astra DB Token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

In [ ]:
# llama-parse is async-first, running the sync code in a notebook
# requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

### Using llama-parse to parse a PDF

In [ ]:
# Grab a PDF from Arxiv for indexing
import requests

# The URL of the file you want to download
url = "https://arxiv.org/pdf/1706.03762.pdf"
# The local path where you want to save the file
file_path = "./attention.pdf"

# Perform the HTTP request
response = requests.get(url, timeout=30)

# Check if the request was successful
if response.status_code == requests.codes.ok:
    # Open the file in binary write mode and save the content
    with open(file_path, "wb") as file:
        file.write(response.content)
    print("Download complete.")
else:
    print("Error downloading the file.")

Download complete.


In [ ]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="text", verbose=True).load_data("./attention.pdf")

Started parsing the file under job_id 50c49013-b7cc-4ed2-8102-499659998b77


In [ ]:
# Take a quick look at some of the parsed text from the document:
documents[0].get_content()[10000:11000]

'                                                    (shifted right)\n                           Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1   Encoder and Decoder Stacks\nEncoder:     The encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected feed-forward network. We employ a residual connection [11] around each of\nthe two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is\nLayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produ

### Storing into Astra DB

In [ ]:
import os

from llama_index.vector_stores.astra_db import AstraDBVectorStore

astra_db_store = AstraDBVectorStore(
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
    collection_name="astra_v_table_llamaparse",
    embedding_dimension=1536,
)

In [ ]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

node_parser = SimpleNodeParser()

nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import ServiceContext, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo"),
    embed_model=OpenAIEmbedding(),
    chunk_size=512,
)

index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

### Simple RAG Example

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=15, service_context=service_context
)

In [ ]:
query = "What is Multi-Head Attention also known as?"

response_1 = query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response_1)


***********New LlamaParse+ Basic Query Engine***********
Multi-Head Attention is also known as an attention function that linearly projects the queries, keys and values multiple times with different, learned linear projections to different dimensions. This allows the attention function to be performed in parallel on each of these projected versions of queries, keys and values.


In [ ]:
# Take a look at one of the source nodes from the response
response_1.source_nodes[0].get_content()

'We used beam search as described in the previous section, but no\ncheckpoint averaging. We present these results in Table 3.\nIn Table 3 rows (A), we vary the number of attention heads and the attention key and value dimensions,\nkeeping the amount of computation constant, as described in Section 3.2.2. While single-head\nattention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.\nIn Table 3 rows (B), we observe that reducing the attention key size dk hurts model quality. This\nsuggests that determining compatibility is not easy and that a more sophisticated compatibility\nfunction than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,\nbigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our\nsinusoidal positional encoding with learned positional embeddings [9], and observe nearly identical\nresults to the base model.\n6.3    English Constituency Parsing\nTo eva

In [ ]:
# Query fails to be answered due to lack of context in Astra DB
query = "What is the color of the sky?"

response_1 = query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response_1)


***********New LlamaParse+ Basic Query Engine***********
The context does not provide information about the color of the sky.
